In [914]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)

2.1.0


In [948]:
data = pd.read_fwf("dataframeEURUSD1.txt")
#new columns
new_columns = ['datetime'] + list(data.columns.values[3:])
a = data.iloc[:,1]
b = data.iloc[:,2]
c = a +' '+ b
data.iloc[:,1] = c
data.drop(index=0,columns=data.columns[[0,2]],inplace=True)
data.reset_index(inplace=True,drop=True)
data.columns = new_columns
data['datetime'] = pd.to_datetime(data.datetime,format='%Y-%m-%d')
data.set_index('datetime', inplace=True)
data.dropna(inplace=True)
#data.drop(data.iloc[:,2],inplace=True)
dataframe = data
print(dataframe.columns)
len(dataframe)


Index(['open', 'close', 'high', 'low', 'volume', 'sma30', 'sma14', 'sma6',
       'macd_fast', 'macd_slow', 'macd_diff', 'macd', 'price_diff', 'rsi14',
       'crossover_macd', 'crossover_sma', 'chopiness', 'band_upper',
       'band_lower', 'stoch_%K', 'v20'],
      dtype='object')


4961

In [981]:
new_data = dataframe.drop(columns=data.columns[[0,4,8,9,14,15,-5,-2,-1]])
new_data.tail().columns


Index(['close', 'sma30', 'sma14', 'sma6', 'macd_diff', 'macd', 'price_diff',
       'rsi14', 'band_upper', 'band_lower'],
      dtype='object')

In [1002]:
g = int(0.9*len(new_data))
h = 0.2*len(new_data)
a = []
b = []
for i in range(len(new_data[:g])):
    if i < 10: continue
    step = new_data[(i-10):i]
    b.append(new_data['price_diff'].shift(-1)[i])
    #step.drop(columns=['price_diff'])
    a.append(step.values)
c = []
d = []
for i in range(len(new_data[g-10:])):
    if i < 10: continue
    i = i - 10 + g
    step = new_data[(i-10):i]
    d.append(new_data['price_diff'].shift(-1)[i])
    #step.drop(columns=['price_diff'])
    c.append(step.values)

a = np.array(a)
b = (np.sign(np.array(b))) + 1
c = np.array(c)
d = (np.sign(np.array(d))) + 1
c = c[:-1]
d = d[:-1]
b = b.astype(int)
d = d.astype(int)

train_images = a
train_labels = b
test_images = c
test_labels = d
b[:10]



array([2, 0, 1, 2, 0, 2, 2, 0, 0, 2])

In [1003]:
l1 = tf.keras.layers.Flatten(input_shape=(20,10))
l2 = tf.keras.layers.Dense(700, activation='relu')
l3 = tf.keras.layers.Dense(230, activation='relu')
l4 = tf.keras.layers.Dense(100, activation='relu')
l5 = tf.keras.layers.Dense(15, activation='relu')
l6 = tf.keras.layers.Dense(3, activation ='softmax')

In [1004]:
model = tf.keras.Sequential([
    l1,
    l2,
    l3,
    l4,
    l5,
    l6
])

In [1005]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [1006]:
model.fit(train_images, train_labels, epochs=10)

Train on 4454 samples
Epoch 1/10
4454/4454 [==============================] - 5s 1ms/sample - loss: 0.9416 - accuracy: 0.4791
Epoch 2/10
4454/4454 [==============================] - 1s 281us/sample - loss: 0.8760 - accuracy: 0.4652
Epoch 3/10
4454/4454 [==============================] - 1s 249us/sample - loss: 0.8811 - accuracy: 0.4762
Epoch 4/10
4454/4454 [==============================] - 1s 240us/sample - loss: 0.8663 - accuracy: 0.4802
Epoch 5/10
4454/4454 [==============================] - 1s 254us/sample - loss: 0.8636 - accuracy: 0.4809
Epoch 6/10
4454/4454 [==============================] - 1s 215us/sample - loss: 0.8628 - accuracy: 0.4744
Epoch 7/10
4454/4454 [==============================] - 1s 164us/sample - loss: 0.8598 - accuracy: 0.4778
Epoch 8/10
4454/4454 [==============================] - 1s 172us/sample - loss: 0.8658 - accuracy: 0.4856
Epoch 9/10
4454/4454 [==============================] - 1s 171us/sample - loss: 0.8601 - accuracy: 0.4897
Epoch 10/10
4454/4454 [===

In [1007]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

496/496 - 0s - loss: 0.8847 - accuracy: 0.4677

Test accuracy: 0.46774194


In [1008]:
preds = model.predict(test_images)
k = [np.argmax(preds[i]) for i in range(len(preds))]
k[-15:]

[2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0]

In [1009]:
test_labels[-15:]

array([2, 0, 0, 2, 0, 0, 2, 2, 2, 1, 2, 0, 2, 0, 2])